#Setup Colab

In [1]:
from google.colab import drive 

drive.mount('/content/gdrive')
%cd "/content/gdrive/My Drive/Colab Notebooks/PBT_MARL_watered_down/"
!pwd
!ls -l

# Install if you haven't done so.
#!pip install tensorflow==2.3.0
!pip install lz4
!pip install 'ray[tune]'
!pip install -U https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.9.0.dev0-cp36-cp36m-manylinux1_x86_64.whl
#!pip install ray[rllib]==0.8.6

#!pip show tensorflow
#!pip show ray
#!cat /etc/os-release

#!rm -rf ~/ray_results/DDPPO_*
#!rm -rf ~/ray_results/PPO_*

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks/PBT_MARL_watered_down
/content/gdrive/My Drive/Colab Notebooks/PBT_MARL_watered_down
total 75
drwx------ 6 root root  4096 Aug 24 09:01 chkpt
-rw------- 1 root root  3139 Jun 11 03:59 Helper.py
-rw------- 1 root root     1 Jun 11 03:59 __init__.py
-rw------- 1 root root  1072 Jun 10 04:55 LICENSE
-rw------- 1 root root  6543 Aug 24 15:06 PBT_MARL.py
-rw------- 1 root root  9014 Jun 10 17:23 pbt_marl_water_down_cpu_only.py
-rw------- 1 root root 35785 Aug 25 07:57 PBT_MARL_water_down.ipynb
drwx------ 2 root root  4096 Aug 24 06:35 __pycache__
drwx------ 2 root root  4096 Aug 25 07:58 ray_results
-rw------- 1 root root  3710 Aug  4 08:18 README.md
-rw------- 1 root root  2056 Aug  4 08:04 RockPaperScissorsEnv.py
  Using cached https://s3-us-west-2.amazonaws.com/ray-wheels/latest/ray-0.9.0.dev0-cp36-cp36m-manylinux1_x86

#Chkpt/restore & log path

In [2]:
g_drive_path = "/content/gdrive/My Drive/Colab Notebooks/PBT_MARL_watered_down/"

local_dir = g_drive_path + "chkpt/"
chkpt_freq = 10
chkpt = 150
restore_path = "{}checkpoint_{}/checkpoint-{}".format(local_dir, chkpt, chkpt)
is_restore = False

log_dir = g_drive_path + "ray_results/"

#Imports

In [3]:
from collections import defaultdict
from typing import Dict
import random
import numpy as np

from gym.spaces import Discrete

import ray
from ray import tune
from ray.tune.logger import pretty_print

from ray.tune.registry import register_env
from ray.rllib.models import ModelCatalog

from ray.rllib.policy import Policy
from ray.rllib.policy.torch_policy import LearningRateSchedule, EntropyCoeffSchedule

from ray.rllib.agents.ppo.ppo_torch_policy import PPOTorchPolicy, KLCoeffMixin, ValueNetworkMixin
from ray.rllib.agents.ppo import ppo
from ray.rllib.agents.ppo.ppo import PPOTrainer
from ray.rllib.agents.ppo import appo
from ray.rllib.agents.ppo.appo import APPOTrainer
from ray.rllib.agents.ppo import ddppo
from ray.rllib.agents.ppo.ddppo import DDPPOTrainer

from ray.rllib.env import BaseEnv
from ray.rllib.env.multi_agent_env import MultiAgentEnv

from ray.rllib.policy.sample_batch import SampleBatch
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.agents.callbacks import DefaultCallbacks

from ray.rllib.utils.schedules import ConstantSchedule
from ray.rllib.utils import try_import_tf
tf = try_import_tf()

from RockPaperScissorsEnv import RockPaperScissorsEnv
from Helper import Helper
from PBT_MARL import PBT_MARL

Instructions for updating:
non-resource variables are not supported in the long term


#Callbacks

In [4]:
"""#Callbacks"""

class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self, worker: RolloutWorker, base_env: BaseEnv,
                         policies: Dict[str, Policy],
                         episode: MultiAgentEpisode, **kwargs):
        #print("on_episode_start {}, _agent_to_policy {}".format(episode.episode_id, episode._agent_to_policy))
        
        for k, v in policies.items():
            key = k + "_gamma"
            episode.user_data[key] = []
            episode.hist_data[key] = []
            #p_0_gamma = worker.get_policy(str(p_id)).config["gamma"]
            #print("p_0_gamma", p_0_gamma)        

        pass

    def on_episode_step(self, worker: RolloutWorker, base_env: BaseEnv,
                        episode: MultiAgentEpisode, **kwargs):
        """
        pole_angle = abs(episode.last_observation_for()[2])
        raw_angle = abs(episode.last_raw_obs_for()[2])
        assert pole_angle == raw_angle
        episode.user_data["pole_angles"].append(pole_angle)
        """
        for k, v in policies.items():
            key = k + "_gamma"
            episode.user_data[key].append(worker.get_policy(k).config["gamma"])

        pass

    def on_episode_end(self, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       **kwargs):
        #print("on_episode_end {}, episode.agent_rewards {}".format(episode.episode_id, episode.agent_rewards))
        """
        pole_angle = np.mean(episode.user_data["pole_angles"])
        print("episode {} ended with length {} and pole angles {}".format(episode.episode_id, episode.length, pole_angle))
        episode.custom_metrics["pole_angle"] = pole_angle
        episode.hist_data["pole_angles"] = episode.user_data["pole_angles"]
        """
        for k, v in policies.items():
            key = k + "_gamma"        
            episode.custom_metrics[key] = np.mean(episode.user_data[key])
            episode.hist_data[key] = episode.user_data[key]


        player_policy = []
        score = []
        for k,v in episode.agent_rewards.items():
            player_policy.append(k)
            score.append(v)

        pol_i_key = player_policy[0][1]
        pol_j_key = player_policy[1][1]
        _, str_i = pol_i_key.split("_")
        _, str_j = pol_j_key.split("_")
        agt_i_key = "agt_" + str_i
        agt_j_key = "agt_" + str_j

        g_helper = ray.get_actor("g_helper")     
        prev_rating_i = ray.get(g_helper.get_rating.remote(agt_i_key))
        prev_rating_j = ray.get(g_helper.get_rating.remote(agt_j_key))
        score_i = score[0]
        score_j = score[1]
        rating_i, rating_j = l_PBT_MARL.compute_rating(prev_rating_i, prev_rating_j, score_i, score_j)
        ray.get(g_helper.update_rating.remote(agt_i_key, agt_j_key, rating_i, rating_j, score_i, score_j))
        #print("on_episode_end ray.get(g_helper.get_agt_store.remote())", ray.get(g_helper.get_agt_store.remote()))

    def on_sample_end(self, worker: RolloutWorker, samples: SampleBatch,
                      **kwargs):
        #print("on_sample_end returned sample batch of size {}".format(samples.count))
        pass

    def on_train_result(self, trainer, result: dict, **kwargs):
        print("trainer.train() result: {} -> {} episodes".format(trainer, result["episodes_this_iter"]))
        # you can mutate the result dict to add new fields to return
        result["callback_ok"] = True
        #print("on_train_result result", result)

        l_PBT_MARL.PBT(trainer)     # perform PBT

        g_helper = ray.get_actor("g_helper")     
        ray.get(g_helper.set_pair.remote())     # set the lastest pair
        #print("on_train_result g_helper.get_pair.remote()", ray.get(g_helper.get_pair.remote()))


        #lr_0 = np.random.rand()
        #lr_1 = lr_0 + 0.1
        #for w in trainer.workers.remote_workers():
            #w.foreach_policy.remote(lambda p, p_id: p.update_lr_schedule(i))  
            #w.for_policy.remote(lambda p: p.update_lr_schedule(lr_0), "p_0")  
            #w.for_policy.remote(lambda p: p.update_lr_schedule(lr_1), "p_1") 


    def on_postprocess_trajectory(
            self, worker: RolloutWorker, episode: MultiAgentEpisode,
            agent_id: str, policy_id: str, policies: Dict[str, Policy],
            postprocessed_batch: SampleBatch,
            original_batches: Dict[str, SampleBatch], **kwargs):
        #print("postprocessed {}, {}, {}, {} steps".format(episode, agent_id, policy_id, postprocessed_batch.count))              
        """
        if "num_batches" not in episode.custom_metrics:
            episode.custom_metrics["num_batches"] = 0
        episode.custom_metrics["num_batches"] += 1
        """
        pass

#Mixin for extending policy & trainer.

In [5]:
class My_Mixin:
    def __init__(self, config):
        self.config = config
        #self.lr_schedule = config["lr_schedule"]
        #self.gamma = config["gamma"]
                
    def update_lr_schedule(self, lr):        
        self.lr_schedule = ConstantSchedule(lr, framework=None)  
        print("update_lr_schedule, lr={}".format(lr))

    def update_gamma(self, gamma):        
        #self.gamma = gamma     # error, policy does not have gamma attribute.
        self.config["gamma"] = gamma      # gamma is only use in postprocess_ppo_gae function in ppo_tf_policy.py so changing config["gamma"] will suffice.
        print("update_gamma, gamma={}".format(gamma))

def setup_mixins(policy, obs_space, action_space, config):
    # Copied from PPO
    ValueNetworkMixin.__init__(policy, obs_space, action_space, config)
    KLCoeffMixin.__init__(policy, config)
    EntropyCoeffSchedule.__init__(policy, config["entropy_coeff"],
                                  config["entropy_coeff_schedule"])
    LearningRateSchedule.__init__(policy, config["lr"], config["lr_schedule"])  
    #My_Mixin.__init__(policy, config)

CustomPolicy = PPOTorchPolicy.with_updates(
    name="Custom_Policy",
    before_init=setup_mixins,
    mixins=[
        LearningRateSchedule, EntropyCoeffSchedule, KLCoeffMixin,
        ValueNetworkMixin, 
        My_Mixin
    ])

def get_policy_class(config):
    return CustomPolicy

CustomTrainer = DDPPOTrainer.with_updates(name="Custom_Trainer",
                                          default_policy=CustomPolicy,
                                          get_policy_class=get_policy_class,
                                          )

#Policy

In [6]:
def init_policies(population_size, obs_space, act_space, use_lstm, hyperparameters_range):
    """
    Sample hyper-parameter from the hyper-parameter distribution.
    """
    policies = {}
    for i in range(population_size):
        pol_key = "p_" + str(i)
        lr = np.random.uniform(low=hyperparameters_range["lr"][0], high=hyperparameters_range["lr"][1], size=None)
        gamma = np.random.uniform(low=hyperparameters_range["gamma"][0], high=hyperparameters_range["gamma"][1], size=None)
        policies[pol_key] = (None, obs_space, act_space, {"model": {"use_lstm": use_lstm},
                                                          "lr": lr,
                                                          "gamma": gamma})
    return policies

def train_policies(population_size):    
    train_policies = []
    for i in range(population_size):
        pol_key = "p_" + str(i)
        train_policies.append(pol_key)

    return policies

def select_policy(agent_id):
    _, i = agent_id.split("_")
    policy = "p_" + str(i)
    #print("select_policy {} {}".format(agent_id , policy))
    return policy     

#Variables

In [7]:
population_size = 6
K = 0.1     
T_select = 0.77 #0.47
binomial_n = 1
inherit_prob = 0.5
perturb_prob = 0.1
perturb_val = [0.8, 1.2]
hyperparameters_range = {"lr": [1e-7, 1e-1], 
                         "gamma": [0.9, 0.999]}

register_env("RockPaperScissorsEnv", lambda _: RockPaperScissorsEnv(_, population_size))     # register RockPaperScissorsEnv with RLlib     
# get obs & act spaces from dummy CDA env
dummy_env = RockPaperScissorsEnv(_, population_size=0)
obs_space = dummy_env.observation_space
act_space = dummy_env.action_space

use_lstm=False
policies = init_policies(population_size, obs_space, act_space, use_lstm, hyperparameters_range)
train_policies = train_policies(population_size)

l_PBT_MARL = PBT_MARL(population_size, 
                      K, T_select, 
                      binomial_n, inherit_prob,
                      perturb_prob, perturb_val)

ray.shutdown()
#ray.init(ignore_reinit_error=True, log_to_driver=True, webui_host='127.0.0.1', num_cpus=2, num_gpus=1)      #start ray
ray.init(ignore_reinit_error=True, log_to_driver=True, num_cpus=2, num_gpus=1)      #start ray
#print("ray.nodes()", ray.nodes())

g_helper = Helper.options(name="g_helper").remote(population_size, policies)      # this object runs on a different ray actor process
ray.get(g_helper.set_pair.remote())

num_iters = 30     # num of main training loop

2020-08-25 07:58:28,101	INFO resource_spec.py:250 -- Starting Ray with 7.13 GiB memory available for workers and up to 3.59 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-08-25 07:58:28,691	INFO services.py:1213 -- View the Ray dashboard at localhost:8265


#Config

In [8]:
def get_config():
    config = ddppo.DEFAULT_CONFIG.copy()

    config["env"] = RockPaperScissorsEnv
    config["multiagent"] = {"policies_to_train": train_policies,
                            "policies": policies,
                            "policy_mapping_fn": select_policy}        
    config["num_cpus_per_worker"] = 0.25                                
    config["num_gpus_per_worker"] = 0.125
    config["num_workers"] = 2      
    config["num_envs_per_worker"] = 3
    config["rollout_fragment_length"] = 30                  
    #config["train_batch_size"] = -1     # must be -1 for DDPPO trainer 
    config["sgd_minibatch_size"] = 10                       
    config["num_sgd_iter"] = 3      # number of epochs to execute per train batch.
    config["callbacks"] = MyCallbacks
    config["log_level"] = "WARN"      # WARN/INFO/DEBUG 
    config["output"] = log_dir

    return config

#Go train

In [9]:
def go_train(config):     
    #trainer = ddppo.DDPPOTrainer(config=config, env="RockPaperScissorsEnv")         
    #trainer = ppo.PPOTrainer(config=config, env="RockPaperScissorsEnv")         
    trainer = CustomTrainer(config=get_config(), env="RockPaperScissorsEnv")         

    if is_restore == True:
        trainer.restore(restore_path) 
    
    result = None
    for i in range(num_iters):
        result = trainer.train()       
        print("training loop = {} of {}".format(i + 1, num_iters))            
        print(pretty_print(result))     # includes result["custom_metrics"]

        #p_0 = trainer.get_policy('p_0')
        #p_0.lr_schedule = ConstantSchedule(0.3, framework=None)

        if i % chkpt_freq == 0:
            checkpoint = trainer.save(local_dir)
            print("checkpoint saved at", checkpoint)
    
    checkpoint = trainer.save(local_dir)
    print("checkpoint saved at", checkpoint)
    

# run everything
go_train(get_config())    

ray.shutdown()

2020-08-25 07:58:32,773	INFO trainer.py:637 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
2020-08-25 07:58:33,197	WARNING worker.py:413 -- ray.get_gpu_ids() will return a list of strings by default in a future version of Ray for compatibility with CUDA. To enable the forward-compatible behavior, use `ray.get_gpu_ids(as_str=True)`.
(pid=2466) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=2466) Instructions for updating:
(pid=2466) non-resource variables are not supported in the long term
(pid=2555) WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=25

trainer.train() result: <ray.rllib.agents.trainer_template.Custom_Trainer object at 0x7fda523aa2b0> -> 18 episodes
(pid=2466) update_lr_schedule, lr=0.05250648179450637
(pid=2466) update_gamma, gamma=0.9128771915578374
(pid=2555) update_lr_schedule, lr=0.05250648179450637
(pid=2555) update_gamma, gamma=0.9128771915578374
(pid=2466) update_lr_schedule, lr=0.02562047833341632
(pid=2466) update_gamma, gamma=0.9549629876590182
(pid=2555) update_lr_schedule, lr=0.02562047833341632
(pid=2555) update_gamma, gamma=0.9549629876590182
(pid=2466) update_lr_schedule, lr=0.020496382666733058
(pid=2466) update_gamma, gamma=0.9549629876590182
(pid=2555) update_lr_schedule, lr=0.020496382666733058
(pid=2555) update_gamma, gamma=0.9549629876590182
(pid=2466) update_lr_schedule, lr=0.020496382666733058
(pid=2466) update_gamma, gamma=0.9549629876590182
(pid=2555) update_lr_schedule, lr=0.020496382666733058
(pid=2555) update_gamma, gamma=0.9549629876590182
(pid=2466) update_lr_schedule, lr=0.0567631112941

(pid=2555) update_gamma, gamma=0.972552746800975
training loop = 9 of 30
callback_ok: true
custom_metrics:
  p_0_gamma_max: 0.9549629876590181
  p_0_gamma_mean: 0.9549629876590179
  p_0_gamma_min: 0.9549629876590181
  p_1_gamma_max: 0.9549629876590181
  p_1_gamma_mean: 0.9549629876590179
  p_1_gamma_min: 0.9549629876590181
  p_2_gamma_max: 0.9774519661959784
  p_2_gamma_mean: 0.9544929772974391
  p_2_gamma_min: 0.9304069342801448
  p_3_gamma_max: 0.9594542998975586
  p_3_gamma_mean: 0.9540091688531439
  p_3_gamma_min: 0.9451726820545104
  p_4_gamma_max: 0.9549629876590181
  p_4_gamma_mean: 0.9387651651398707
  p_4_gamma_min: 0.9214855724021866
  p_5_gamma_max: 0.9549629876590181
  p_5_gamma_mean: 0.9505620399295401
  p_5_gamma_min: 0.9109535103642414
date: 2020-08-25_07-59-08
done: false
episode_len_mean: 10.0
episode_reward_max: 0.0
episode_reward_mean: 0.0
episode_reward_min: 0.0
episodes_this_iter: 18
episodes_total: 162
experiment_id: 992f5ff3caa745479dead671b835b523
hostname: f8dd